In [1]:
import numpy as np
import random 
import pandas as pd #pliki CSV
import seaborn as sns #charts
sns.set_palette('husl')

In [2]:
iris = pd.read_csv('iris.csv')

In [3]:
class DataProcessing:
    @staticmethod
    def shuffle(X):
        for i in range(len(X)-1,0,-1):
            j=random.randint(0,i)
            X.iloc[i], X.iloc[j] = X.iloc[j], X.iloc[i]
        return X

    @staticmethod
    def splitSet(X):
        XTrain = X.iloc[:int(len(X)*0.7),:]
        XVal = X.iloc[int(len(X)*0.7):,:]
        return XTrain, XVal

    @staticmethod
    def normalize(X):
        for c in range(4):# ['sepal.length','sepal.width','petal.length','petal.width']:
            mn=min(X.iloc[:,c])
            mx=max(X.iloc[:,c])
            for i in range(len(X)):
                X.iloc[i,c]=(X.iloc[i,c].copy()-mn)/(mx-mn)
        return X

In [4]:
irisMixed=DataProcessing.shuffle(iris)
irisMixed=DataProcessing.normalize(irisMixed)

In [5]:
irisTrain, irisVal = DataProcessing.splitSet(irisMixed)

In [6]:
class Neuron:
    weights=[] #for output synapses
    
    def __init__(self):
        self.weights=[]
            
    def activationFunction(this,x):
        return np.tanh(x)
    def fire(this,inputValue):
        output=[]
        for x in range(len(this.weights)):
            output.append(this.weights[x]*this.activationFunction(inputValue))
        return output
    
    #def generateWeights():
        
    def loadWeights(this,wagi):
        this.weights=wagi.copy()

class NeuralNetwork:
    layers=[]
    def __init__(self):
        self.layers=[]
    
    def addLayer(this,n):
        this.layers.append([])
        for i in range(n):
            this.layers[len(this.layers)-1].append(Neuron())
            
    def loadWeights(this,weights): #weights count == synapses count
        i=0
        for j in range(len(this.layers)-1):
            for x in range(len(this.layers[j])):
                tmp=[]
                for y in range(len(this.layers[j+1])):
                    tmp.append(weights[i])
                    i+=1
                this.layers[j][x].loadWeights(tmp)
                
    def countSynapses(this):
        count=0
        for i in range(1,len(this.layers)):
            count+=(len(this.layers[i])*len(this.layers[i-1]))
        return count

    def calculateError(this,training):
        var={'Setosa':[1,0,0],'Virginica':[0,1,0],'Versicolor':[0,0,1]}
        errors=[]
        for x in range(len(training)):
            result=this.feedforward(training.iloc[x])
            error=0
            for i in range(3):
                error+=(result[i]-var[training.iloc[x].variety][i])**2
            errors.append(error)
        return sum(errors)/len(training)
                
    def feedforward(this,sample):
        layerInput=[]
        for x in range(len(this.layers[0])):
            layerInput.append(sample[x])
        for i in range(len(this.layers)-1):
            layerOutput=[]
            for x in range(len(this.layers[i+1])):
                layerOutput.append(0)
            for x in range(len(layerInput)):
                tmp=this.layers[i][x].fire(layerInput[x])
                for x in range(len(tmp)):
                    layerOutput[x]+=tmp[x]
            layerInput=layerOutput.copy()
        Result=[]
        for x in range(len(layerInput)):
            Result.append(this.layers[len(this.layers)-1][x].activationFunction(layerInput[x]))
        return Result
    
class Particle:
    actualPosition=[]
    v=[]
    xbl=[]
    xblError=0
    
    def __init__(self,n):
        self.actualPosition=[]
        self.v=[]
        self.xbl=[]
        for i in range(n):
            self.actualPosition.append(random.uniform(0,1))
            self.v.append(0.1)
            
    def move(this,w,phi1,phi2,xbg):
        for i in range(len(this.actualPosition)):
            this.v[i]=w*this.v[i]+phi1*random.uniform(0,1)*(this.xbl[i]-this.actualPosition[i])+phi2*random.uniform(0,1)*(xbg[i]-this.actualPosition[i])
            this.actualPosition[i]+=this.v[i]
            
    def checkBestPosition(this,dataset,network):
        network.loadWeights(this.actualPosition)
        error=network.calculateError(dataset)
        if this.xblError>error:
            this.xblError=error
            this.xbl=this.actualPosition.copy()
    
class PSO:
    particles=[]
    xbg=[]
    xbgError=0
    
    def __init__(self):
        self.particles=[]
        self.xbg=[]
        self.xbgError=0
        
    #phi1+phi2==1
    def run(this,T,n,phi1,phi2, network, dataset):
        errors=[]
        synapsesCount=network.countSynapses()
        for i in range(n):
            this.particles.append(Particle(synapsesCount))
            this.particles[i].xbl=this.particles[i].actualPosition.copy()
            network.loadWeights(this.particles[i].xbl)
            this.particles[i].xblError=network.calculateError(dataset)
            errors.append(this.particles[i].xblError)
            
        this.sortParticles(errors,n)
        this.xbg=this.particles[0].actualPosition.copy()
        this.xbgError=this.particles[0].xblError.copy()
        
        for t in range(T):
            errors=[]
            for i in range(n):
                this.particles[i].move(1,phi1,phi2,this.xbg)
                this.particles[i].checkBestPosition(dataset,network)
                network.loadWeights(this.particles[i].actualPosition)
                errors.append(network.calculateError(dataset))
            this.sortParticles(errors,n)
            if(errors[0]<this.xbgError):
                this.xbgError=errors[0]
                this.xbg=this.particles[0].xbl.copy()
        return this.xbg

    def sortParticles(this,errors,n):
        for i in range(n):
            for j in range(n-i-1):
                if errors[j]>errors[j+1]:
                    errors[j],errors[j+1]=errors[j+1],errors[j]
                    this.particles[j],this.particles[j+1]=this.particles[j+1],this.particles[j]
    

In [35]:
nn=NeuralNetwork()
pso=PSO()

nn.addLayer(4)
nn.addLayer(8)
nn.addLayer(16)
nn.addLayer(8)
nn.addLayer(3)

In [ ]:
bestWeights=pso.run(200,10,0.7,0.3,nn,irisTrain)
nn.loadWeights(bestWeights)

In [37]:
correct=0
var={'Setosa':[1,0,0],'Virginica':[0,1,0],'Versicolor':[0,0,1]}
for x in range(len(irisVal)):
    result=nn.feedforward(irisVal.iloc[x])
    res=[0,0,0]
    res[result.index(max(result))]=1
    if(res==var[irisVal.iloc[x].variety]):
        correct+=1
print(100*correct/len(irisVal))

75.55555555555556


In [11]:
def zapisWag(bestWeights):
    zapis=open("wagi.txt","w")
    for x in bestWeights:
        zapis.write(str(x)+'\n')
    zapis.close()
    
def wczytywanieWag():
    wczytywanie=open("wagi.txt","r")
    wagi=[]
    wagi=wczytywanie.read().split("\n")
    wagi.pop(len(wagi)-1)
    for i in range(len(wagi)):
        wagi[i]=float(wagi[i])
    wczytywanie.close()
    return wagi

In [12]:
zapisWag(bestWeights)

In [36]:
wagi=wczytywanieWag()
nn.loadWeights(wagi)

In [ ]:
#wagi dla 75.(5)%
wagi=[0.018211515163520917,
-0.25770000381617214,
0.21600300180901286,
0.5015855837508785,
-0.5346824988607243,
0.7698067369922482,
0.3701164483453237,
0.6714294242250919,
-0.40474249224766234,
0.051233127731658404,
0.3201053774554417,
0.22361946871384056,
0.022147448653026827,
-0.5919192535186759,
-0.5170463997535177,
0.3356748791227182,
0.1787090104246794,
0.47264505607702423,
-0.5255410453399133,
0.3626578698602049,
-0.7171016006966187,
0.1226486493960654,
0.4952512938203997,
-0.004692311370619363,
-0.5790737047040797,
-0.19746759337497913,
0.21040977948572645,
-0.1635695356339167,
-0.08621430734992663,
-0.09643692352869711,
-0.49925147780209045,
0.3293493754556808,
0.2927112820365803,
-0.34269007575110727,
0.7805059224603695,
0.19116220108074816,
-0.14409602187019907,
-0.02513719005593834,
-0.37502586490264733,
0.4145938379029734,
-0.03506799397466964,
0.0019873604122454605,
0.27349316924411105,
-0.37304222017412464,
-0.31252471567291157,
-0.11656251547793148,
0.2629171152179428,
0.19840820923767716,
-0.34436152869438125,
0.16831332780798025,
0.13601081009378357,
0.39688908787941357,
0.8147522666832708,
0.22666301524103538,
0.058588611997606135,
-0.031599319143850675,
0.2974884968068899,
-0.1206925663141985,
0.08351968286209366,
-0.08578009753755637,
0.7596696001097578,
0.7052038103336795,
0.5050596692002032,
0.08005085822242917,
-0.15890006540107304,
-0.6411006023067264,
0.5600899369003126,
0.9737945723178933,
-0.5409124532510592,
-0.01858450512358784,
-0.3981279406168426,
-0.11858683892325028,
0.43784113556913773,
-0.6988001773141953,
-0.4252409540612007,
0.3245848115910907,
0.34253336582724736,
0.3305131179036865,
-0.03152444577643507,
0.1000443588612255,
-0.6674416997522291,
-0.8893754237541243,
-0.3378310944601859,
0.4498373728933166,
-0.7045832603531901,
0.23371569676348392,
0.22039326438266113,
0.4936606201538352,
-0.5844247980915129,
-0.03645113063471339,
-0.45752634113054697,
0.319707586693256,
-0.519810412286162,
-0.11806841259553288,
0.14617247818337376,
-0.24900387488765288,
0.27495434037191946,
-1.1104498546919377,
-0.0022731881884311966,
-0.3123111475490409,
0.16105573136296525,
0.654310403324198,
0.8518330999691207,
0.2110183660103589,
0.2902752171818453,
-0.31024456844387877,
0.05907594311858272,
0.5707807114347099,
-0.051061694593722215,
-0.2687389882205083,
-0.2086057646610711,
-0.853367128567044,
0.6988043442606954,
-0.1269105025222442,
0.9299165223284875,
0.289599935526272,
-0.4224289564927997,
0.020734201321289104,
0.04089510914646893,
0.13314629726935626,
-0.23241708261262012,
0.025739689400468155,
0.2332850874399136,
0.03356475078259136,
0.6922079172262211,
0.26148194513519873,
0.03401148223514334,
0.37716730671016374,
-0.039435270165362635,
-0.031975084528422106,
-0.178203140451759,
0.025338877377280883,
-0.3025313373643927,
0.48968700769869433,
0.24720825188807266,
0.21316476606021062,
-0.69956331529888,
-0.17555283637019445,
0.2552767163042546,
0.18605629337772733,
-0.48045518499311857,
-0.2644936632002572,
0.0018449438815670433,
-0.15951856805878328,
-0.11795031097090769,
-0.013383270842520723,
0.0187801051036527,
0.1894749884160541,
-0.3751722620004263,
0.14322129765379504,
-0.16411428596015395,
-0.05605171326948838,
0.40832463534890234,
-0.2799718700190332,
0.2522643229144746,
-0.387955277188236,
-0.30747632078216686,
0.9542744122828208,
-0.6086822417782214,
0.3842529422677758,
0.5549023019757162,
-0.04443311935222669,
-0.2179621136168552,
-0.37317055599405136,
0.40943868691723756,
0.36175108613929235,
0.26723349287958664,
-0.2703424243182805,
0.4816348718225403,
-0.34543345132454967,
0.3924570931327095,
-0.14013810094349582,
0.052797534133365534,
0.3354168259577077,
-0.08594962471285413,
-0.5058184288449433,
0.342781816135628,
-0.14127081459457425,
0.04495604393346625,
0.16116079585621076,
0.7706805414843387,
-0.11184462885394458,
-0.31681880979306387,
-0.4290252916130602,
-0.04798513867416963,
-0.20111996887222466,
-0.5866510064021262,
-0.5446105128650418,
-0.1822110363447741,
-0.22216553731159677,
0.11433540383004039,
-0.0060433309395200065,
-0.30144589835470725,
0.36655322119269,
-0.44674231401070447,
-0.35904824295720367,
0.30224583034426644,
0.20751231417644248,
0.3216754546654498,
-0.15553097604726798,
0.06586910306673623,
0.7108656446408159,
0.4891697744472573,
-0.38934885585210066,
0.3817939330215345,
0.12600307114596088,
0.10406058611841132,
-0.04570660522176803,
0.1653167752888996,
0.9182078600181947,
0.28917183499293714,
0.38195677525782723,
0.49379712590960817,
-0.2941793529257342,
0.6501084517912719,
0.3732858596796994,
-0.3096521791251671,
-0.2835664065886919,
0.22240858160289806,
-0.4255143730352815,
0.34161839795341487,
0.07459728624464482,
0.11784239282217637,
-0.3268950779626515,
-0.24630323929810186,
-0.11758058634859715,
0.33151874255295527,
-0.2855139220286965,
0.8486162101218373,
0.06096482489136289,
0.18793701902711618,
-0.027296068734251822,
0.9641393883641121,
0.26989850129335263,
0.04101970130658299,
-0.2236600031403875,
0.07692625827297758,
0.28465455933129813,
-0.20671233036246273,
-0.041675166086491494,
-0.25299398826341674,
-0.6817361331171059,
0.6250481846330086,
0.9027284638191665,
0.32585273700425615,
-0.047914525481696765,
-0.3004916343169854,
0.24845998497553481,
0.4131030140932328,
-0.28217547240952734,
-0.7757740543703561,
0.3759325087778348,
-0.02629033626857027,
-0.7286247930741344,
-0.5301796119369513,
-0.27254648239181734,
-0.17326178035993356,
-0.8252960833637231,
-0.2079982008125561,
0.6188121995676183,
0.5137387514094831,
-0.03155945318155895,
0.19059621190889364,
-0.506014837669575,
0.36454744290127644,
0.0533451379620728,
-0.2303611794854532,
-0.28502787932202567,
0.23482557978418644,
-0.8149487103141828,
0.058987838877492774,
-0.39564272251188615,
-0.05389914139822148,
0.1983785278479839,
-0.5286087086404199,
-0.44799804956269745,
-0.4504149250938685,
-0.01814133726541478,
-0.09792115465796022,
-0.44300318121502413,
-0.13860780557659497,
0.30919692681225774,
-0.12531142989025784,
0.2851364431481821,
0.0669281985162035,
-0.3830864234743985,
0.3348460889431889,
-0.41053021579542015,
-1.1687358224453979,
0.5940464219334528,
-1.026632662671468,
0.0956325963348468,
-0.4804071750496144,
-0.11509381460843818,
-0.6493847413711868,
-0.25671946475322005,
-0.2879234686833463,
-0.05824501304469324,
-0.09469313174401611,
0.26610147860764477,
-0.26719913435131865,
-0.2668363241423478,
-0.576749672362342,
0.05249922523093962,
-0.09488512507245979,
0.3288210491188691,
0.05571551421787793,
-0.34153424255661247,
0.009349282411701802,
-0.021907514198934358,
0.19830381434892005,
-0.9171291141517753]
nn.loadWeights(wagi)